In [0]:
import pandas as pd
import imutils
import cv2
import numpy as np
from matplotlib import pyplot as plt
import json
from skimage import color
from skimage.feature import hog
from skimage import data, exposure

In [0]:
data = pd.read_csv(r'input.csv')

In [4]:
data.head()

,emotion (label),pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,0,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,0,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,0,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
data.shape

(35887, 3)

In [6]:
training_data = data[data['Usage'] == 'Training']
training_data.shape

(28709, 3)

In [7]:
testing_data = data[data['Usage'] != 'Training']
testing_data.shape

(7178, 3)

In [8]:
training_data = training_data.iloc[:,:-1]
training_data.shape

(28709, 2)

In [9]:
testing_data = testing_data.iloc[:,:-1]
testing_data.shape

(7178, 2)

In [0]:
#data1 = np.array(data.iloc[:,1]).astype('uint8')
#img_length = 48
#data1 = data1.reshape(-1,3,img_length,img_length).transpose([0,2,3,1])
def pixel_extraction(pixels):
    tor = []
    new_pixels = []
    for i in pixels:
        #new_pixel = []
        tor =[]
        i = i.split(' ')
        #print(i)
        #break
        for j in i:
            #(len(tor) == 3):
            # new_pixel.append(tor)
            # tor = []
      #     j= int(j)
            j= np.uint8(j)
            tor.append(j)
      # new_pixels.append(np.array(new_pixel))
    
        new_pixels.append(tor)
    return(new_pixels)

In [11]:
pixel_train = pixel_extraction(training_data.iloc[:,1])
pixel_train = np.array(pixel_train)
pixel_train.shape

(28709, 2304)

In [12]:
pixel_test = pixel_extraction(testing_data.iloc[:,1])
pixel_test = np.array(pixel_test)
pixel_test.shape

(7178, 2304)

In [0]:
def shape_conversion(new_pixels):
    genuine = []
    for i in new_pixels:
        genuine.append(i.reshape(48,48))
#new_pixels[0] = new_pixels[0].reshape(48,48)
    genuine = np.array(genuine)
    return(genuine)

In [14]:
pixel_train = shape_conversion(pixel_train)
pixel_train.shape

(28709, 48, 48)

In [15]:
pixel_test = shape_conversion(pixel_test)
pixel_test.shape

(7178, 48, 48)

In [19]:
ppc = 8
hog_images_train = []
hog_features_train = []
for image in pixel_train:
    fd,hog_image = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
    hog_images_train.append(hog_image)
    hog_features_train.append(fd)

/usr/local/lib/python3.6/dist-packages/skimage/feature/_hog.py:239: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


In [20]:
print(len(hog_features_train[3]))

1152


In [21]:
ppc = 8
hog_images_test= []
hog_features_test = []
for image in pixel_test:
    fd1,hog_image1 = hog(image, orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualise=True)
    hog_images_test.append(hog_image1)
    hog_features_test.append(fd1)

/usr/local/lib/python3.6/dist-packages/skimage/feature/_hog.py:239: skimage_deprecation: Argument `visualise` is deprecated and will be changed to `visualize` in v0.16
  'be changed to `visualize` in v0.16', skimage_deprecation)


In [0]:
train_labels =  np.array(training_data.iloc[:,0]).reshape(len(training_data.iloc[:,0]),1)
test_labels =  np.array(testing_data.iloc[:,0]).reshape(len(testing_data.iloc[:,0]),1)

In [28]:
train_labels.shape

(28709, 1)

In [29]:
train_labels

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [0]:
from sklearn import svm
from sklearn.metrics import classification_report,accuracy_score
clf = svm.SVC()
hog_features_train = np.array(hog_features_train)
data_frame_1 = np.hstack((hog_features_train,train_labels))
np.random.shuffle(data_frame_1)

In [0]:
hog_features_test = np.array(hog_features_test)
data_frame_2 = np.hstack((hog_features_test,test_labels))
np.random.shuffle(data_frame_2)

In [38]:
data_frame_1[:,-1]

array([0., 0., 0., ..., 0., 0., 1.])

In [0]:
x_train, x_test = data_frame_1[:,:-1],  data_frame_2[:,:-1]
y_train, y_test = data_frame_1[:,-1:].ravel() , data_frame_2[:,-1:].ravel()


In [40]:
clf.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [0]:
y_pred = clf.predict(x_test)

In [42]:
print("Accuracy: ",(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy:  0.7528559487322374


              precision    recall  f1-score   support

         0.0       0.75      1.00      0.86      5404
         1.0       0.00      0.00      0.00      1774

    accuracy                           0.75      7178
   macro avg       0.38      0.50      0.43      7178
weighted avg       0.57      0.75      0.65      7178



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
import csv
#with open('Train_HOG_features.csv','w') as f:
 # f.csv.write(data_frame_1)
pd.DataFrame(data_frame_1).to_csv("Train_HOG_features.csv")
 

In [0]:
pd.DataFrame(data_frame_2).to_csv("Test_HOG_features.csv")

In [0]:
data_frame_3 = np.hstack((y_pred,[i for i in range(0,len(testing_data.iloc[:,0]))]))
pd.DataFrame(data_frame_3).to_csv("Predictions.csv")